# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [14]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))

print(adult_dt)


       age          workclass  fnlwgt    education  education-num  \
0       39          State-gov   77516    Bachelors             13   
1       50   Self-emp-not-inc   83311    Bachelors             13   
2       38            Private  215646      HS-grad              9   
3       53            Private  234721         11th              7   
4       28            Private  338409    Bachelors             13   
...    ...                ...     ...          ...            ...   
32556   27            Private  257302   Assoc-acdm             12   
32557   40            Private  154374      HS-grad              9   
32558   58            Private  151910      HS-grad              9   
32559   22            Private  201490      HS-grad              9   
32560   52       Self-emp-inc  287927      HS-grad              9   

            marital-status          occupation    relationship    race  \
0            Never-married        Adm-clerical   Not-in-family   White   
1       Married-civ-spo

# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 


+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?



*(Comment here.)*

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Define the numerical and categorical features
numerical_features = [
    'age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'
]
categorical_features = [
    'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'
]

# Create a pipeline for preprocessing the numerical features
numerical_pipeline = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),  # KNN imputation
    ('scaler', RobustScaler())  # Robust scaling
])

# Create a pipeline for preprocessing the categorical features
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute using the most frequent value (mode)
    ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first'))  # One-hot encoding with dropping binary variable column
])

# Create the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),  # Process numerical features
        ('cat', categorical_pipeline, categorical_features)  # Process categorical features
    ])

# Example: Fit and transform the training data (X_train)
X_train_transformed = preprocessor.fit_transform(X_train)

# Example: Transform the test data (X_test)
X_test_transformed = preprocessor.transform(X_test)

# Optionally, you can print the shapes of the transformed data to check
print("Transformed X_train shape:", X_train_transformed.shape)
print("Transformed X_test shape:", X_test_transformed.shape)


Transformed X_train shape: (22792, 100)
Transformed X_test shape: (9769, 100)


## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [ ]:
# Create the pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  # Apply the preprocessing step from the previous section
    ('classifier', RandomForestClassifier(random_state=42))  # Add a RandomForestClassifier as the classifier
])

# Fit the model on the training data
model_pipeline.fit(X_train, Y_train)

# Make predictions on the test set
y_pred = model_pipeline.predict(X_test)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Print accuracy
print(f"Accuracy: {accuracy_score(Y_test, y_pred)}")

# Print classification report
print("\nClassification Report:")
print(classification_report(Y_test, y_pred))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(Y_test, y_pred))


Accuracy: 0.8555635172484389

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.93      0.91      7455
           1       0.73      0.63      0.67      2314

    accuracy                           0.86      9769
   macro avg       0.81      0.78      0.79      9769
weighted avg       0.85      0.86      0.85      9769


Confusion Matrix:
[[6907  548]
 [ 863 1451]]


# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [11]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Define the scoring metrics for cross-validation
scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

# Perform cross-validation using the pipeline
cv_results = cross_validate(model_pipeline, X, Y, cv=5, scoring=scoring, return_train_score=True)

# Display the training and validation results
print("Training results:")
print(f"Mean Negative Log Loss: {cv_results['train_neg_log_loss'].mean():.4f}")
print(f"Mean ROC AUC: {cv_results['train_roc_auc'].mean():.4f}")
print(f"Mean Accuracy: {cv_results['train_accuracy'].mean():.4f}")
print(f"Mean Balanced Accuracy: {cv_results['train_balanced_accuracy'].mean():.4f}")

print("\nValidation results:")
print(f"Mean Negative Log Loss: {cv_results['test_neg_log_loss'].mean():.4f}")
print(f"Mean ROC AUC: {cv_results['test_roc_auc'].mean():.4f}")
print(f"Mean Accuracy: {cv_results['test_accuracy'].mean():.4f}")
print(f"Mean Balanced Accuracy: {cv_results['test_balanced_accuracy'].mean():.4f}")

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Training results:
Mean Negative Log Loss: -0.0803
Mean ROC AUC: 1.0000
Mean Accuracy: 1.0000
Mean Balanced Accuracy: 1.0000

Validation results:
Mean Negative Log Loss: -0.3698
Mean ROC AUC: 0.9031
Mean Accuracy: 0.8548
Mean Balanced Accuracy: 0.7753


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [ ]:
# Get fold-level results from cross-validation
fold_results = pd.DataFrame(cv_results)

# Sort the results by negative log loss of the test set (validation set)
fold_results_sorted = fold_results.sort_values(by='test_neg_log_loss', ascending=True)

# Display the sorted fold-level results
fold_results_sorted[['test_neg_log_loss', 'test_roc_auc', 'test_accuracy', 'test_balanced_accuracy']]


,test_neg_log_loss,test_roc_auc,test_accuracy,test_balanced_accuracy
1,-0.413153,0.897008,0.851044,0.769948
0,-0.367746,0.903050,0.854598,0.770004
4,-0.365701,0.907273,0.857033,0.780424
2,-0.352683,0.906301,0.853655,0.778853
3,-0.349777,0.902032,0.857494,0.777244


Calculate the mean of each metric. 

In [ ]:
# Calculate the mean of each metric
mean_metrics = fold_results[['test_neg_log_loss', 'test_roc_auc', 'test_accuracy', 'test_balanced_accuracy']].mean()

# Display the mean values
print(mean_metrics)

test_neg_log_loss        -0.369812
test_roc_auc              0.903133
test_accuracy             0.854765
test_balanced_accuracy    0.775295
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [12]:

# Make predictions on the test set
Y_pred = model_pipeline.predict(X_test)  # Predicted labels
Y_pred_proba = model_pipeline.predict_proba(X_test)  # Predicted probabilities

# Calculate the performance metrics
neg_log_loss_value = log_loss(Y_test, Y_pred_proba)  # Negative log loss
roc_auc_value = roc_auc_score(Y_test, Y_pred_proba[:, 1])  # ROC AUC (for the positive class)
accuracy_value = accuracy_score(Y_test, Y_pred)  # Accuracy
balanced_accuracy_value = balanced_accuracy_score(Y_test, Y_pred)  # Balanced Accuracy

# Store the results in a dictionary
performance_metrics = {
    "Negative Log Loss": neg_log_loss_value,
    "ROC AUC": roc_auc_value,
    "Accuracy": accuracy_value,
    "Balanced Accuracy": balanced_accuracy_value
}

# Display the results
print(performance_metrics)


{'Negative Log Loss': 0.3807704552987868, 'ROC AUC': 0.9021627025187715, 'Accuracy': 0.8555635172484389, 'Balanced Accuracy': 0.776772504807004}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

It is convenient because it transforms the target variable 'income' into a binary format, which is an essential preprocessing step for classification problems in machine learning. It creates a binary target for a binary classification, ensures compatibility sckit-learn, better model performance. For ROC AUC target label of interest should be encoded as 1 for the positive class. This is important because the AUC score is computed by plotting the true positive rate (sensitivity) against the false positive rate at various thresholds.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.